In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import warnings
warnings.filterwarnings("ignore")   #To avoid the warnings

### 1. Predicting the Cost Using Linear Regression Model

In [4]:
Task2=pd.read_csv(r"C:\Users\ganig\OneDrive\Desktop\Dataset\Manpower_SF_Nov12.csv")

In [5]:
Task2.drop(columns=["plan_cd","dept","division", "direct_indirect",
                 "employee_class","workforce_category_descr","attend_reason_code",
                 "plant_line_running"],inplace=True)
Task2.rename(columns={'plant_code':'classes','absentee':'cost','total_attend_hours':'hours',
                  'jobtitle':'job','attend_type_code':'code'},inplace=True)


In [6]:
Task2.head()

,Unnamed: 0,classes,attend_date,job,code,cost,hours
0,0,AEP,2022-12-09,PA,ABS,1,8.0
1,1,AEP,2023-11-10,LSS,ABS,1,8.0
2,2,AEP,2023-06-19,PA,PTO,1,8.0
3,3,AEP,2022-03-25,PS,PTO,1,8.0
4,4,MAP,2023-02-13,PS,PTO,1,8.0


In [7]:
Task2.count()

Unnamed: 0     396881
classes        396881
attend_date    396881
job            396881
code           396881
cost           396881
hours          396881
dtype: int64

In [8]:
#sorting the date
start_date = ('2023-10-01')
end_date =('2023-11-30')
Task3=Task2[(Task2['attend_date'] >= start_date) & (Task2['attend_date'] <= end_date)]

In [9]:
Task3.count()

Unnamed: 0     19596
classes        19596
attend_date    19596
job            19596
code           19596
cost           19596
hours          19596
dtype: int64

In [10]:
#Used to convert the categorical data into numerical data
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Task3['classes'] = label_encoder.fit_transform(Task3['classes'])      

In [11]:
label_encoder = LabelEncoder()
Task3['job'] = label_encoder.fit_transform(Task3['job'])   

In [12]:
label_encoder = LabelEncoder()
Task3['code'] = label_encoder.fit_transform(Task3['code']) 

In [13]:
Task3["classes"].value_counts()

classes
0    5070
1    4398
4    3754
3    2754
2    2414
6     792
5     414
Name: count, dtype: int64

In [14]:
Task3["job"].value_counts()

job
2    14615
0     2303
5     1253
3      505
4      456
1      322
6      142
Name: count, dtype: int64

In [15]:
Task3["code"].value_counts()

code
0    11483
3     7874
1      209
2       30
Name: count, dtype: int64

In [16]:
Task3

,Unnamed: 0,classes,attend_date,job,code,cost,hours
1,1,1,2023-11-10,0,0,1,8.0
18,19,0,2023-10-05,2,3,1,4.0
42,45,1,2023-10-25,2,0,1,8.0
54,57,0,2023-10-31,2,0,5,40.0
59,62,4,2023-10-23,2,0,9,72.0
...,...,...,...,...,...,...,...
396799,22274,1,2023-10-06,4,3,1,8.0
396804,22279,4,2023-10-24,0,0,1,8.0
396848,22330,1,2023-10-12,2,3,2,16.0
396853,22336,6,2023-10-13,3,0,1,8.0


In [17]:
Task3['attend_date'] = pd.to_datetime(Task3['attend_date'])

# Extracting year, month, and day into separate columns
Task3['year'] = Task3['attend_date'].dt.year
Task3['month'] = Task3['attend_date'].dt.month
Task3['day'] = Task3['attend_date'].dt.day

In [18]:
Task3

,Unnamed: 0,classes,attend_date,job,code,cost,hours,year,month,day
1,1,1,2023-11-10,0,0,1,8.0,2023,11,10
18,19,0,2023-10-05,2,3,1,4.0,2023,10,5
42,45,1,2023-10-25,2,0,1,8.0,2023,10,25
54,57,0,2023-10-31,2,0,5,40.0,2023,10,31
59,62,4,2023-10-23,2,0,9,72.0,2023,10,23
...,...,...,...,...,...,...,...,...,...,...
396799,22274,1,2023-10-06,4,3,1,8.0,2023,10,6
396804,22279,4,2023-10-24,0,0,1,8.0,2023,10,24
396848,22330,1,2023-10-12,2,3,2,16.0,2023,10,12
396853,22336,6,2023-10-13,3,0,1,8.0,2023,10,13


### Splitting the data

In [19]:
x=Task3[["classes","job","code","hours","year","month","day"]]

In [20]:
y=Task3["cost"]

In [21]:
x

,classes,job,code,hours,year,month,day
1,1,0,0,8.0,2023,11,10
18,0,2,3,4.0,2023,10,5
42,1,2,0,8.0,2023,10,25
54,0,2,0,40.0,2023,10,31
59,4,2,0,72.0,2023,10,23
...,...,...,...,...,...,...,...
396799,1,4,3,8.0,2023,10,6
396804,4,0,0,8.0,2023,10,24
396848,1,2,3,16.0,2023,10,12
396853,6,3,0,8.0,2023,10,13


In [22]:
y

1         1
18        1
42        1
54        5
59        9
         ..
396799    1
396804    1
396848    2
396853    1
396860    1
Name: cost, Length: 19596, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.3)   #test_size=30%

In [24]:
x_train.count()   #70%

classes    13717
job        13717
code       13717
hours      13717
year       13717
month      13717
day        13717
dtype: int64

In [25]:
y_test.count()   #30%

5879

### Importing the required model

In [26]:
from sklearn.linear_model import LinearRegression

In [27]:
lr=LinearRegression()

### Training the model

In [28]:
lr.fit(x_train,y_train)

LinearRegression()

In [29]:
y_pred=lr.predict(x_test)

In [30]:
y_pred

array([ 9.00335272,  1.01709098,  1.0205861 , ..., 20.07996399,
        1.02044111,  1.12717084])

### Score

In [31]:
#from sklearn.metrics import accuracy_score
#lr = accuracy_score(y_test, y_pred)

In [32]:
lr.score(x_test,y_test)

0.9981470506446538

### MSE(Mean Squared Error)

In [33]:
from sklearn.metrics import accuracy_score,mean_squared_error
mean_squared_error(y_test,y_pred)

0.062456980482353756

### 2. Predicting the hours Using Linear Regression Model

In [34]:
Task3.head()

,Unnamed: 0,classes,attend_date,job,code,cost,hours,year,month,day
1,1,1,2023-11-10,0,0,1,8.0,2023,11,10
18,19,0,2023-10-05,2,3,1,4.0,2023,10,5
42,45,1,2023-10-25,2,0,1,8.0,2023,10,25
54,57,0,2023-10-31,2,0,5,40.0,2023,10,31
59,62,4,2023-10-23,2,0,9,72.0,2023,10,23


### Splitting thhe data

In [35]:
a=Task3[["classes","job","code","cost","year","month","day"]]
b=Task3["hours"]

In [36]:
a

,classes,job,code,cost,year,month,day
1,1,0,0,1,2023,11,10
18,0,2,3,1,2023,10,5
42,1,2,0,1,2023,10,25
54,0,2,0,5,2023,10,31
59,4,2,0,9,2023,10,23
...,...,...,...,...,...,...,...
396799,1,4,3,1,2023,10,6
396804,4,0,0,1,2023,10,24
396848,1,2,3,2,2023,10,12
396853,6,3,0,1,2023,10,13


In [37]:
b

1          8.0
18         4.0
42         8.0
54        40.0
59        72.0
          ... 
396799     8.0
396804     8.0
396848    16.0
396853     8.0
396860     8.0
Name: hours, Length: 19596, dtype: float64

In [38]:
from sklearn.model_selection import train_test_split
x1_train,x2_test,y1_train,y2_test =train_test_split(a,b,test_size=0.3,random_state=0)   #test_size=30%
#the term "random_state" is often used to set a seed for the random number generator.

In [39]:
x1_train.count()

classes    13717
job        13717
code       13717
cost       13717
year       13717
month      13717
day        13717
dtype: int64

In [40]:
x2_test.count()

classes    5879
job        5879
code       5879
cost       5879
year       5879
month      5879
day        5879
dtype: int64

### Importing the required model

In [41]:
from sklearn.linear_model import LinearRegression
model=LinearRegression()

### Training the model 

In [42]:
model.fit(x1_train,y1_train)

LinearRegression()

In [43]:
Y_Pred=model.predict(x2_test)

In [44]:
Y_Pred

array([ 7.88234945, 32.19581041,  7.84427538, ...,  7.84316344,
       15.00060864,  7.05502366])

### Score

In [45]:
#from sklearn.metrics import accuracy_score
#accuracy_score(y2_test, Y_Pred)
model.score(x2_test,y2_test)

0.9984421276084692

In [46]:
#model.score(x1_train,y1_train)

### MSE(Mean Squared Error)

In [47]:
from sklearn.metrics import accuracy_score,mean_squared_error
mean_squared_error(y2_test,Y_Pred)

3.2815908494718222